Генерация данных из двух файлов с данными дампа ДБ в формате SQL.

* flats.txt - содержит информацию о квартирах
* pairs.txt - содержит инфу об одинаковых объявлениях


Формат данных flats.txt:

1. index
2. offer_id, 
3. description, 
4. geo, 
5. building, 
6. roomscount, 
7. floornumber, 
8. category, 
9. totalarea, 
10. userid, 
11. publisheduserid, 
12. flattype, 
13. bargainterms

Формат данных pairs.txt:

1. index, 
2. offer_id1, 
3. offer_id2, 
4. is_test, 
5. resolution


In [1]:
with open('flats.txt', 'rt', encoding='utf-8-sig') as f:
    lines = f.readlines()

Вывод данных для анализа

In [2]:
lines[0].split('\t')

['0',
 '3dddd44ec9b2d96c12d9929a1baf060a',
 'Мы работаем, и у нас безопасно!\\n- Заселяем клиентов круглосуточно и дистанционно, без встречи с сотрудником\\n- Принимаем оплату бесконтактным способом\\n- Дезинфицируем все поверхности в квартире перед заселением и после выезда\\n- Останавливаясь у нас, Вы никогда не будете обмануты или введены в заблуждение \\n- Фото соответствует действительности \\n- У нас широкий выбор апартаментов. Все квартиры имеют удобное расположение, находятся рядом с остановками, магазинами и тп.\\n- Мы предоставляем надежные отчетные документы (возможно без проживания)\\n- Работаем с юридическими лицами. \\n\\nУютная, теплая и чистая двухкомнатная квартира в самом центре Втузгородка по ул. Мира 37 недалеко от центра города, ТРЦ Комсомолл ждет своих гостей на СУТКИ, ЧАСЫ, НЕДЕЛИ, МЕСЯЦ !\\n\\nРядом: УрФУ Мира 19, Институт МЧС Мира 22, Юридическая Академия Комсомольская 21, Суворовское училище, Дендропарк, ЦК "Урал" Студенческая 3, Гуманитарный университет Студе

Генерируем данные из строки. В данном случае игнорируем некоторые данные, так как время ограничено.

In [13]:
import json
import string

data = {}
fail_count = 0 
success_count = 0
for il, l in enumerate(lines):
    try:
        l = l.replace("\'", "\"").replace("True", "\"True\"").replace("False", "\"False\"")
        t = l.split('\t')
        s = t[2].replace("\\n", "")
        s = s.lower()
        s = s.translate(str.maketrans('', '', string.punctuation))
        data[t[1]] = {
            "description": s,
            "geo": json.loads(t[3].replace('\n', "")),
            "building": json.loads(t[4].replace('\n', "")),
            "roomscount": t[5],
            "floornumber": t[6],
            "category": t[7],
            "totalarea": t[8], 
            "userid": t[9], 
            "publisheduserid":  t[10], 
            "flattype": t[11],
            "bargainterms": json.loads(t[12].replace('\n', ""))
        }
        success_count += 1
    except Exception as e:
        fail_count += 1
        
print(success_count, fail_count)

19049 217


In [14]:
with open('pairs.txt', 'rt', encoding='utf-8-sig') as f:
    lines = f.readlines()
pair = [l.split('\t') for l in lines]

После анализа данных было решено в начале выбрать небольшое количество параметром которые показались самыми важными.

In [16]:
flat = ['description', 'roomscount', 'floornumber', 'category', 'totalarea', 'flattype', 'userid', 'publisheduserid']
geo = ['userInput']
building = ['floorsCount', 'totalArea']
bargainterms = ['price']

In [17]:
new_data = {}
for k in data:
    row = [data[k][f] for f in flat]
    row.append(data[k]['geo']['userInput'].lower().translate(str.maketrans('', '', string.punctuation)))
    row.append(data[k]['building']['floorsCount'] if 'floorsCount' in data[k]['building'] else 0)
    row.append(data[k]['building']['totalArea'] if 'totalArea' in data[k]['building'] else 0)
    row.append(data[k]['bargainterms']['price'])
    new_data[k] = row

И теперь удалим те пары, для которых нет квартир.

In [18]:
all_keys = list(new_data.keys())

In [19]:
X_pairs = []
y = []

In [20]:
all_pairs = [p[1] + p[2] for p in pair ] + [p[2] + p[1] for p in pair]

In [21]:
for p in pair:
    if p[1] in new_data and p[2] in new_data:
        X_pairs.append([ new_data[p[1]], new_data[p[2]] ])
        y.append(1)

У нас есть позитивные примеры. Теперь нужно сгенерировать негативные. В данном случае сделаем в пять раз больше негативных. Понятно, что в реальности негативных примеров будет в сотни раз больше, но для первой итерации этого должно быть достаточно.

In [22]:
import random

generate = 5

for k in all_keys:
    for _ in range(generate):
        candidate = random.choice(all_keys)
        if candidate != k and k+candidate not in all_pairs:
            X_pairs.append([new_data[k], new_data[candidate]])
            y.append(0)

Сохраняем данные.

In [23]:
import pickle

with open("data_5.pkl", 'wb') as f:
    pickle.dump((X_pairs, y), f)